### 1. Iz Annotations/train.json filtrirati samo ona pitanja koja ne zahtevaju prepoznavanje teksta (izlazi van domena rešenja)
### 2. Podeliti filtrirani skup na train i test deo (data/train.json i data/test.json)

### Napomena 1 - Annotations/val.json i Annotations/test.json necemo koristiti ni za obucavanje ni za evaluaciju zato sto nemamo informacije o tome da li zahtevaju prepoznavanje teksta
### Napomena 2 - Ravnomerna podela na osnovu answer_type (number, yes/no, other, unanswerable)

In [117]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [118]:
ocr = pd.read_csv('data/ocr.csv')
ocr.head()

,IMG,QSN,TXT,OBJ,COL,CNT,OTH
0,VizWiz_train_00000000.jpg,What's the name of this product?,4,3,0,0,0
1,VizWiz_train_00000001.jpg,Can you tell me what is in this can please?,2,5,1,1,0
2,VizWiz_train_00000002.jpg,Is this enchilada sauce or is this tomatoes? ...,4,3,1,0,0
3,VizWiz_train_00000003.jpg,What is the captcha on this screenshot?,4,2,1,1,0
4,VizWiz_train_00000004.jpg,What is this item?,3,5,1,0,0


In [119]:
ocr.TXT.describe()

count    14239.000000
mean         2.107732
std          1.819784
min          0.000000
25%          0.000000
50%          2.000000
75%          4.000000
max          5.000000
Name: TXT, dtype: float64

In [120]:
ocr_filtered = ocr[ocr.TXT <= 1]
ocr_filtered.head()

,IMG,QSN,TXT,OBJ,COL,CNT,OTH
5,VizWiz_train_00000005.jpg,What's this?,1,5,1,0,0
8,VizWiz_train_00000008.jpg,What color do these look?,1,3,4,0,0
9,VizWiz_train_00000009.jpg,Surface look clean? Thank you.,0,5,1,0,0
10,VizWiz_train_00000010.jpg,Is this.,1,5,2,0,0
11,VizWiz_train_00000012.jpg,what is this?,0,5,2,0,0


In [121]:
ocr_filtered.TXT.describe()

count    6241.000000
mean        0.273514
std         0.445798
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: TXT, dtype: float64

In [122]:
data = pd.read_json('Annotations/train.json')

In [123]:
data_filtered = data[data.image.isin(ocr_filtered.IMG)]
data_filtered.head()

,image,question,answers,answer_type,answerable
5,VizWiz_train_00000005.jpg,What's this?,"[{'answer_confidence': 'yes', 'answer': 'boots...",other,1
8,VizWiz_train_00000008.jpg,What color do these look?,"[{'answer_confidence': 'yes', 'answer': 'orang...",other,1
9,VizWiz_train_00000009.jpg,Surface look clean? Thank you.,"[{'answer_confidence': 'yes', 'answer': 'yes'}...",yes/no,1
10,VizWiz_train_00000010.jpg,Is this.,"[{'answer_confidence': 'maybe', 'answer': 'and...",other,1
12,VizWiz_train_00000012.jpg,what is this?,"[{'answer_confidence': 'yes', 'answer': 'remot...",other,1


In [124]:
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6238 entries, 5 to 19924
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        6238 non-null   object
 1   question     6238 non-null   object
 2   answers      6238 non-null   object
 3   answer_type  6238 non-null   object
 4   answerable   6238 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 292.4+ KB


### Zašto ocr_filtered ima 6241, a data_filtered 6238 elementara? nzm

In [125]:
data_filtered.answer_type.describe()

count      6238
unique        4
top       other
freq       5678
Name: answer_type, dtype: object

In [126]:
data_filtered.answer_type.unique()

array(['other', 'yes/no', 'number', 'unanswerable'], dtype=object)

In [127]:
train, test = train_test_split(data_filtered, train_size=0.9, stratify=data_filtered.answer_type)
train.to_json('data/train.json')
test.to_json('data/test.json')

In [128]:
train = pd.read_json('data/train.json')
train.head()

,image,question,answers,answer_type,answerable
1616,VizWiz_train_00001616.jpg,What is it?,"[{'answer_confidence': 'yes', 'answer': 'rug'}...",other,1
11910,VizWiz_train_00011910.jpg,WHAT DO YOU SEE?,"[{'answer_confidence': 'maybe', 'answer': 'leg...",other,1
13392,VizWiz_train_00013392.jpg,What is this?,"[{'answer_confidence': 'no', 'answer': 'pendan...",other,1
7274,VizWiz_train_00007274.jpg,What do I see,"[{'answer': 'toilet paper', 'answer_confidence...",other,1
15488,VizWiz_train_00015488.jpg,What color are these socks?,"[{'answer_confidence': 'yes', 'answer': 'black...",other,1


In [129]:
test = pd.read_json('data/test.json')
test.head()

,image,question,answers,answer_type,answerable
7824,VizWiz_train_00007824.jpg,What color is this?,"[{'answer_confidence': 'maybe', 'answer': 'blu...",other,1
7288,VizWiz_train_00007288.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'cream...",other,1
6489,VizWiz_train_00006489.jpg,What is this?,"[{'answer_confidence': 'yes', 'answer': 'mug'}...",other,1
9376,VizWiz_train_00009376.jpg,what color is the shirt,"[{'answer_confidence': 'yes', 'answer': 'white...",other,1
6591,VizWiz_train_00006591.jpg,What are these pills?,"[{'answer_confidence': 'yes', 'answer': 'wefwe...",other,1


In [130]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5614 entries, 1616 to 4966
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        5614 non-null   object
 1   question     5614 non-null   object
 2   answers      5614 non-null   object
 3   answer_type  5614 non-null   object
 4   answerable   5614 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 263.2+ KB


In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 7824 to 11965
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   image        624 non-null    object
 1   question     624 non-null    object
 2   answers      624 non-null    object
 3   answer_type  624 non-null    object
 4   answerable   624 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 29.2+ KB
